# Clasificación de Especies de Iris con Redes Neuronales (Keras) y Regularización

**Disciplina:** Aprendizaje Profundo, Redes Neuronales, Clasificación, Keras (TensorFlow)

**Objetivo:**
El objetivo de este notebook es construir, entrenar y evaluar una red neuronal para clasificar las especies de flores del dataset Iris utilizando Keras. Se incorporarán técnicas de preprocesamiento, regularización (L2 y Dropout) y callbacks de Keras (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau) para mejorar el entrenamiento y la robustez del modelo.

## 1. Carga de Librerías y Configuración Inicial

**Propósito de esta sección:**
Importar todas las bibliotecas necesarias y configurar el entorno para el análisis, incluyendo la fijación de semillas para reproducibilidad.

**Bibliotecas Clave:**
* **`numpy`, `pandas`**: Para manipulación de datos.
* **`matplotlib.pyplot`, `seaborn`**: Para visualizaciones.
* **`sklearn.datasets`**: Para cargar el dataset Iris.
* **`sklearn.model_selection`**: Para `train_test_split`.
* **`sklearn.preprocessing`**: Para `StandardScaler` y `LabelEncoder` (aunque `to_categorical` maneja bien las clases numéricas).
* **`sklearn.metrics`**: Para `classification_report` y `confusion_matrix`.
* **`tensorflow.keras`**: Para construir y entrenar la red neuronal.

In [ ]:
# Comandos mágicos de IPython (opcional en scripts)
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Importación de bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Configuración para reproducibilidad
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Configuración de estilo y visualización
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

## 2. Funciones Personalizadas

### Descripción de la Función: `cargar_y_preparar_datos_iris`

**Objetivo Principal:**
Cargar el dataset Iris, realizar preprocesamiento básico (escalado de características, codificación one-hot del objetivo) y dividirlo en conjuntos de entrenamiento y prueba.

**Características:**
* **Procesamiento:**
    1. Carga el dataset Iris.
    2. Crea un DataFrame de Pandas para exploración (opcional).
    3. Separa características (X) y objetivo (y).
    4. Escala las características X usando `StandardScaler`.
    5. Convierte las etiquetas y a formato categórico (one-hot encoding).
    6. Divide los datos en conjuntos de entrenamiento y prueba.
* **Valor de Retorno:**
    * `X_train, X_test, y_train, y_test`: Conjuntos de datos divididos y preprocesados.
    * `scaler`: El objeto `StandardScaler` ajustado (para posible uso en datos nuevos).
    * `feature_names`, `target_names`: Nombres de características y clases.

In [ ]:
def cargar_y_preparar_datos_iris(test_size=0.2, random_state=SEED):
    """
    Carga, preprocesa y divide el dataset Iris.
    """
    print("Cargando y preparando el dataset Iris...")
    iris = load_iris()
    X = iris.data
    y = iris.target
    feature_names = iris.feature_names
    target_names = iris.target_names

    # Opcional: Explorar con Pandas
    df = pd.DataFrame(X, columns=feature_names)
    df['species_code'] = y
    df['species_name'] = df['species_code'].map({i: name for i, name in enumerate(target_names)})
    print("\nPrimeras filas del dataset Iris:")
    print(df.head())
    print("\nDistribución de clases:")
    print(df['species_name'].value_counts())

    # Escalar características
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Codificación One-Hot para el objetivo
    y_categorical = to_categorical(y)

    # Dividir datos
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_categorical, test_size=test_size, random_state=random_state, stratify=y # stratify para clasificación
    )
    
    print(f"\nDimensiones: X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}")
    return X_train, X_test, y_train, y_test, scaler, feature_names, target_names

### Descripción de la Función: `crear_modelo_clasificacion_keras`

**Objetivo Principal:**
Definir y compilar un modelo de red neuronal secuencial con Keras para clasificación.

**Características:**
* **Entrada:**
    * `input_dim` (int): Número de características de entrada.
    * `output_dim` (int): Número de clases de salida.
* **Procesamiento:**
    1. Crea un modelo `Sequential`.
    2. Añade capas `Dense` con activación 'relu', regularización L2 y Dropout.
       (Considerar `BatchNormalization` opcionalmente).
    3. Añade una capa de salida `Dense` con activación 'softmax'.
    4. Compila el modelo con optimizador 'adam', pérdida 'categorical_crossentropy' y métrica 'accuracy'.
* **Valor de Retorno:**
    * `model` (tf.keras.Model): El modelo Keras compilado.

In [ ]:
def crear_modelo_clasificacion_keras(input_dim, output_dim, l2_lambda=0.001, dropout_rate=0.3):
    """
    Crea un modelo de red neuronal para clasificación con Keras.
    """
    print("\nCreando el modelo de clasificación con Keras...")
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,), kernel_regularizer=regularizers.l2(l2_lambda)),
        # BatchNormalization(), # Opcional: puede ayudar a estabilizar/acelerar
        Dropout(dropout_rate),
        Dense(32, activation='relu', kernel_regularizer=regularizers.l2(l2_lambda)),
        # BatchNormalization(), # Opcional
        Dropout(dropout_rate),
        Dense(output_dim, activation='softmax') # Capa de salida para clasificación multiclase
    ])

    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    print("\nResumen del modelo:")
    model.summary()
    return model

### Descripción de la Función: `graficar_historial_entrenamiento`

**Objetivo Principal:**
Graficar las curvas de pérdida y precisión del entrenamiento y validación.

**Características:**
* **Entrada:**
    * `history` (tf.keras.callbacks.History): Objeto devuelto por `model.fit()`.
* **Procesamiento:**
    * Extrae las métricas del historial.
    * Crea subplots para la precisión y la pérdida.
* **Salida:** Muestra los gráficos.

In [ ]:
def graficar_historial_entrenamiento(history):
    """
    Grafica la precisión y la pérdida durante el entrenamiento y validación.
    """
    print("\nGraficando historial de entrenamiento...")
    acc = history.history.get('accuracy')
    val_acc = history.history.get('val_accuracy')
    loss = history.history.get('loss')
    val_loss = history.history.get('val_loss')
    epochs_range = range(len(acc if acc else val_acc if val_acc else loss if loss else val_loss)) # Manejar si alguna métrica no está

    plt.figure(figsize=(14, 5))

    if acc and val_acc:
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, acc, label='Precisión (Entrenamiento)')
        plt.plot(epochs_range, val_acc, label='Precisión (Validación)')
        plt.legend(loc='lower right')
        plt.title('Precisión de Entrenamiento y Validación')
        plt.xlabel('Épocas')
        plt.ylabel('Precisión')

    if loss and val_loss:
        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, loss, label='Pérdida (Entrenamiento)')
        plt.plot(epochs_range, val_loss, label='Pérdida (Validación)')
        plt.legend(loc='upper right')
        plt.title('Pérdida de Entrenamiento y Validación')
        plt.xlabel('Épocas')
        plt.ylabel('Pérdida')
    
    if not (acc and val_acc and loss and val_loss): # Si faltan métricas para ambos gráficos
        if acc and val_acc: # Solo hay precisión
             plt.title('Precisión de Entrenamiento y Validación')
             plt.xlabel('Épocas'); plt.ylabel('Precisión')
        elif loss and val_loss: # Solo hay pérdida
            plt.title('Pérdida de Entrenamiento y Validación')
            plt.xlabel('Épocas'); plt.ylabel('Pérdida')

    plt.show()

### Descripción de la Función: `evaluar_y_visualizar_clasificacion`

**Objetivo Principal:**
Evaluar el modelo de clasificación en el conjunto de prueba y visualizar la matriz de confusión.

**Características:**
* **Entrada:**
    * `model` (tf.keras.Model): Modelo Keras entrenado.
    * `X_test` (np.ndarray): Características del conjunto de prueba.
    * `y_test_cat` (np.ndarray): Etiquetas one-hot del conjunto de prueba.
    * `target_names` (list): Nombres de las clases.
* **Procesamiento:**
    1. Evalúa el modelo para obtener pérdida y precisión en el test set.
    2. Realiza predicciones y las convierte de one-hot a etiquetas de clase.
    3. Imprime el reporte de clasificación.
    4. Genera y muestra la matriz de confusión.
* **Salida:** Muestra métricas y gráficos.

In [ ]:
def evaluar_y_visualizar_clasificacion(model, X_test, y_test_cat, target_names):
    """
    Evalúa el modelo de clasificación y visualiza la matriz de confusión.
    """
    print("\nEvaluando el modelo en el conjunto de prueba...")
    loss, accuracy = model.evaluate(X_test, y_test_cat, verbose=0)
    print(f"Pérdida en el conjunto de prueba: {loss:.4f}")
    print(f"Precisión en el conjunto de prueba: {accuracy:.4f}")

    # Predicciones
    y_pred_proba = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred_proba, axis=1)
    y_true_classes = np.argmax(y_test_cat, axis=1)

    # Reporte de Clasificación
    print("\nReporte de Clasificación:")
    print(classification_report(y_true_classes, y_pred_classes, target_names=target_names, zero_division=0))

    # Matriz de Confusión
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
    plt.title('Matriz de Confusión')
    plt.xlabel('Predicción')
    plt.ylabel('Real')
    plt.show()

## 3. Desarrollo del Ejercicio: Clasificación de Iris con Keras

### 3.1. Carga y Preparación de Datos

Cargamos el dataset Iris, lo escalamos y lo dividimos en conjuntos de entrenamiento y prueba.

In [ ]:
X_train_iris, X_test_iris, y_train_iris, y_test_iris, scaler_iris, iris_feature_names, iris_target_names = \
    cargar_y_preparar_datos_iris()

### 3.2. Creación del Modelo de Clasificación

Definimos la arquitectura de nuestra red neuronal usando Keras.

In [ ]:
input_dim_iris = X_train_iris.shape[1]
output_dim_iris = y_train_iris.shape[1] # Número de clases (después de to_categorical)

modelo_iris = crear_modelo_clasificacion_keras(input_dim_iris, output_dim_iris, l2_lambda=0.005, dropout_rate=0.25)

### 3.3. Entrenamiento del Modelo

Entrenamos el modelo, utilizando callbacks para mejorar el proceso.
* `EarlyStopping`: Detiene el entrenamiento si la pérdida de validación no mejora tras `patience` épocas.
* `ModelCheckpoint`: Guarda el modelo con el mejor rendimiento en el conjunto de validación.
* `ReduceLROnPlateau`: Reduce la tasa de aprendizaje si la pérdida de validación se estanca.

In [ ]:
print("\nIniciando el entrenamiento del modelo Iris...")

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
model_checkpoint = ModelCheckpoint('best_iris_model.keras', monitor='val_loss', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=0.00001, verbose=1)

callbacks_list = [early_stopping, model_checkpoint, reduce_lr]

# Entrenamiento
history_iris = modelo_iris.fit(
    X_train_iris, y_train_iris,
    epochs=200, # Un número alto de épocas, EarlyStopping se encargará
    batch_size=8, # Lotes pequeños para Iris
    validation_split=0.2, # Usar una porción de los datos de entrenamiento para validación interna
    callbacks=callbacks_list,
    verbose=1 # 0 = silent, 1 = progress bar, 2 = one line per epoch.
)

# Cargar el mejor modelo guardado por ModelCheckpoint (si restore_best_weights=False en EarlyStopping)
# Si restore_best_weights=True en EarlyStopping, esto no es estrictamente necesario ya que el modelo ya tiene los mejores pesos.
# modelo_iris = keras.models.load_model('best_iris_model.keras')

### 3.4. Visualización del Historial de Entrenamiento

Observamos cómo evolucionaron la precisión y la pérdida durante el entrenamiento.

In [ ]:
if history_iris:
    graficar_historial_entrenamiento(history_iris)

### 3.5. Evaluación del Modelo

Evaluamos el rendimiento del modelo entrenado en el conjunto de prueba.

In [ ]:
if modelo_iris:
    evaluar_y_visualizar_clasificacion(modelo_iris, X_test_iris, y_test_iris, iris_target_names)

## 4. Conclusiones del Ejercicio (Clasificación Iris)

**Resumen de Hallazgos:**
* Se cargó y preprocesó el dataset Iris, escalando las características y aplicando codificación one-hot a las etiquetas.
* Se construyó una red neuronal secuencial con Keras, incorporando:
    * Capas `Dense` con activación ReLU.
    * Regularización L2 (`kernel_regularizer`) para penalizar pesos grandes.
    * Capas `Dropout` para reducir el sobreajuste.
    * Una capa de salida `softmax` adecuada para clasificación multiclase.
* El modelo fue compilado con el optimizador `adam` y la función de pérdida `categorical_crossentropy`.
* Durante el entrenamiento, se utilizaron callbacks:
    * `EarlyStopping` para detener el entrenamiento prematuramente si no había mejora, previniendo el sobreajuste.
    * `ModelCheckpoint` para guardar la mejor versión del modelo basada en `val_loss`.
    * `ReduceLROnPlateau` para ajustar la tasa de aprendizaje dinámicamente.
* La precisión final alcanzada en el conjunto de prueba fue de **[Completar con la precisión obtenida, ej: 0.97]**.
* El reporte de clasificación y la matriz de confusión mostraron que el modelo fue capaz de **[Describir brevemente el rendimiento por clase, ej: clasificar correctamente la mayoría de las instancias, con algunas confusiones menores entre 'versicolor' y 'virginica']**.
* Las curvas de aprendizaje (pérdida y precisión vs. épocas) indicaron **[Describir si hubo sobreajuste, si EarlyStopping actuó, etc.]**.

**Aprendizaje General:**
Este ejercicio demostró cómo construir una red neuronal para clasificación multiclase con Keras, aplicando buenas prácticas como el preprocesamiento de datos, la regularización para combatir el sobreajuste, y el uso de callbacks para un entrenamiento más eficiente y efectivo. La combinación de estas técnicas permite desarrollar modelos más robustos y con mejor capacidad de generalización.

*(Nota: Los resultados específicos deben completarse después de ejecutar completamente el notebook.)*